In [1]:

from facenet_pytorch import  InceptionResnetV1, training
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
from utility.custom_dataset import CustomDataset
from utility.custom_dataloader import CustomDataLoader
from models.custvit import CustVIT
from utility.helper import Helper

/home/adil/anaconda3/envs/fr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Checking and loading CUDA if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
# Usage
root_directory = 'Dataset'  # Path to your dataset
loader = CustomDataset(root_dir=root_directory)

dataset = loader.get_dataset()
print(loader.get_classes())

Running on device: cuda:0
['Abdullah Munir', 'Aizaz', 'Ali riaz', 'Anas', 'Anousha', 'Ayaz', 'Fahad', 'Faizan', 'Farhan', 'Hammad', 'Hannan', 'Haris', 'Hirar', 'Irfan', 'Kashmala', 'Khadija', 'Kumail', 'Maryam', 'Moiz', 'Moiz SaP', 'Moizullah', 'Mustafa', 'Noor', 'Obaid', 'Raheel', 'Saqib', 'Sufyan', 'Talal', 'Tayyaba', 'Usaid', 'Zeeshan', 'Zehra', 'ahmed', 'ahsan', 'asad', 'basit', 'bilal', 'gopi', 'musaddiq', 'muzammil', 'omer', 'osama', 'owais', 'rafay', 'rafia', 'shehroz', 'shezil', 'sohaib', 'subhan', 'vinay', 'zain', 'zainab', 'zamran', 'zayan']


['Abdullah Munir', 'Aizaz', 'Ali riaz', 'Anas', 'Anousha', 'Ayaz', 'Fahad', 'Faizan', 'Farhan', 'Hammad', 'Hannan', 'Haris', 'Hirar', 'Irfan', 'Kashmala', 'Khadija', 'Kumail', 'Maryam', 'Moiz', 'Moiz SaP', 'Moizullah', 'Mustafa', 'Noor', 'Obaid', 'Raheel', 'Saqib', 'Sufyan', 'Talal', 'Tayyaba', 'Usaid', 'Zeeshan', 'Zehra', 'ahmed', 'ahsan', 'asad', 'basit', 'bilal', 'gopi', 'musaddiq', 'muzammil', 'omer', 'osama', 'owais', 'rafay', 'rafia', 'shehroz', 'shezil', 'sohaib', 'subhan', 'vinay', 'zain', 'zainab', 'zamran', 'zayan']


In [9]:
# Initialize the custom data loader
loader = CustomDataLoader(dataset=dataset, train_ratio=0.9, batch_size=8, shuffle=True)

# Access the train and test DataLoaders
trainloader = loader.get_trainloader()
testloader = loader.get_testloader()

In [12]:
from PIL import Image
for data in testloader:
    img, label = data
    print(img.mode)

TypeError: expected np.ndarray (got numpy.ndarray)

In [9]:
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [11]:
# InceptionResnetV1 Model

models={}
model = InceptionResnetV1(
    classify=True,# ?
    pretrained='vggface2',
    num_classes=len(dataset.classes)
)

model=model.to(device)
models["InceptionResnetV1"]=model
model = CustVIT(
    num_classes=len(dataset.classes)
    )

model=model.to(device)

models["VIT"]=model

In [10]:

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])
loss_fn = torch.nn.CrossEntropyLoss()

NameError: name 'model' is not defined

In [11]:
# Apply transfer learning modifications 
key='InceptionResnetV1'

#load the previous model if exist otherwise initialize the new model


try:
    model, optimizer, start_epoch=Helper. load_checkpoint(model, optimizer,Helper.get_checkpoint_path(key=key))
    model = model.to(device)
    total_epochs = start_epoch
except Exception as  ex   :
    start_epoch=0
    total_epochs = 0
    print(f"new model loaded:{ex}")
    
model = Helper.transfer_learning(models[key],key,Helper.get_checkpoint_path(key=key),device=device)
model=model.to(device)


new model loaded:name 'model' is not defined


NameError: name 'models' is not defined

In [12]:


metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [13]:
model.eval()
training.pass_epoch(
    model, loss_fn, testloader,
    batch_metrics=metrics, show_running=True, device=device,
)

NameError: name 'model' is not defined

In [14]:
#Training loop with tensorboard summary writer
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
epochs = 100



print(f'\n\nModel:{key} before Training')
print('-' * 10)
model.eval()
training.pass_epoch(
    model, loss_fn, testloader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)




for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    #training loop
    model.train()
    training.pass_epoch(
        model, loss_fn, trainloader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    #testing loop
    model.eval()
    training.pass_epoch(
        model, loss_fn, testloader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
    total_epochs += 1
    Helper.save_checkpoint(model, optimizer, total_epochs, file_path=Helper.get_checkpoint_path(key))
writer.close()



Model:InceptionResnetV1 before Training
----------


NameError: name 'model' is not defined